In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
import urllib as urlb
import time
pd.set_option('display.max.columns', 100)
# to draw pictures in jupyter notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv('adult.csv', sep=",")
user_device = pd.read_csv('user_device.csv', sep=",")
user_usage = pd.read_csv('user_usage.csv', sep=",")
android_device = pd.read_csv('android_devices.csv', sep=",")

In [3]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [4]:
data['gender'].value_counts()

Male      32650
Female    16192
Name: gender, dtype: int64

In [5]:
data.loc[data['gender'] == 'Female', 'age'].mean()

36.92798913043478

In [6]:
float((data['native-country'] == 'Germany').sum())/data.shape[0]

0.00421768150362393

In [7]:
ages1 = data.loc[data['income'] == '>50K', 'age']
ages2 = data.loc[data['income'] == '<=50K', 'age']
print("The average age of the rich: {0} +- {1} years, poor - {2} +- {3} years.".format(round(ages1.mean()), round(ages1.std(), 1), round(ages2.mean()), round(ages2.std(), 1)))

The average age of the rich: 44 +- 10.6 years, poor - 37 +- 14.1 years.


In [8]:
data.loc[data['income'] == '>50K', 'education'].unique()

array(['Assoc-acdm', 'Some-college', 'Prof-school', 'HS-grad', 'Masters',
       'Doctorate', 'Bachelors', 'Assoc-voc', '9th', '10th', '7th-8th',
       '11th', '5th-6th', '1st-4th', '12th', 'Preschool'], dtype=object)

In [9]:
for (race, sex), sub_df in data.groupby(['race', 'gender']):
    print("Race: {0}, gender: {1}".format(race, sex))
    print(sub_df['age'].describe())

Race: Amer-Indian-Eskimo, gender: Female
count    185.000000
mean      36.237838
std       12.840056
min       17.000000
25%       26.000000
50%       35.000000
75%       46.000000
max       80.000000
Name: age, dtype: float64
Race: Amer-Indian-Eskimo, gender: Male
count    285.000000
mean      36.989474
std       11.703943
min       17.000000
25%       29.000000
50%       35.000000
75%       44.000000
max       82.000000
Name: age, dtype: float64
Race: Asian-Pac-Islander, gender: Female
count    517.000000
mean      35.657640
std       12.637799
min       17.000000
25%       25.000000
50%       34.000000
75%       44.000000
max       81.000000
Name: age, dtype: float64
Race: Asian-Pac-Islander, gender: Male
count    1002.000000
mean       38.994012
std        12.824878
min        18.000000
25%        29.000000
50%        37.000000
75%        46.000000
max        90.000000
Name: age, dtype: float64
Race: Black, gender: Female
count    2308.000000
mean       37.905979
std        12.7360

In [10]:
data.loc[(data['gender'] == 'Male') &
     (data['marital-status'].isin(['Never-married', 'Separated', 'Divorced','Widowed'])), 'income'].value_counts()

<=50K    11414
>50K      1001
Name: income, dtype: int64

In [11]:
data.loc[(data['gender'] == 'Male') &
     (data['marital-status'].str.startswith('Married')), 'income'].value_counts()

<=50K    11318
>50K      8917
Name: income, dtype: int64

In [12]:
data['marital-status'].value_counts()

Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: marital-status, dtype: int64

In [13]:
max_load = data['hours-per-week'].max()
print("max hours per week =",(max_load))
num_workaholics = data[data['hours-per-week'] == max_load].shape[0]
print("workaholics =",(num_workaholics))
rich = float(data[(data['hours-per-week'] == max_load) & (data['income'] == '>50K')].shape[0]) / num_workaholics
print("Percent of rich among workaholics = ",(rich))

max hours per week = 99
workaholics = 137
Percent of rich among workaholics =  0.29927007299270075


In [14]:
pd.crosstab(data['native-country'], data['income'], values=data['hours-per-week'], aggfunc=np.mean).T

native-country,?,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,France,Germany,Greece,Guatemala,Haiti,Holand-Netherlands,Honduras,Hong,Hungary,India,Iran,Ireland,Italy,Jamaica,Japan,Laos,Mexico,Nicaragua,Outlying-US(Guam-USVI-etc),Peru,Philippines,Poland,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
income,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
<=50K,39.552590,41.157895,37.378151,36.686047,39.123457,39.201923,41.561224,37.948718,35.819444,39.375000,40.090909,38.898649,41.870968,38.788235,36.409091,40.0,34.055556,39.00,34.076923,38.044944,39.972973,41.269231,38.690141,38.626374,39.733333,39.333333,39.931416,36.847826,41.409091,36.214286,38.014286,37.228571,41.254545,38.908537,40.833333,41.052632,35.307692,43.04,38.8,38.817189,37.848101,38.533333
>50K,45.318182,43.888889,46.126984,42.027778,56.250000,42.852941,42.800000,47.833333,43.454545,46.297872,46.500000,45.706897,55.555556,36.666667,40.666667,NaN,50.000000,43.75,46.333333,46.274194,47.954545,45.181818,45.647059,42.400000,47.062500,40.000000,45.638298,38.333333,40.000000,40.000000,43.588235,39.588235,46.750000,39.900000,46.666667,51.400000,45.538462,53.00,40.0,45.471853,39.428571,43.375000


In [15]:
(data['workclass'] == "private").value_counts()

False    48842
Name: workclass, dtype: int64

In [16]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [17]:
def example1_pandasql(data):
    aggr_query = '''
        SELECT 
            avg("hours-per-week") as average_hours_per_week,
            income
        FROM data 
        GROUP BY income
        '''
    return ps.sqldf(aggr_query, locals()).set_index('income')

In [18]:
weekday_engagement = example1_pandasql(data)
weekday_engagement

,average_hours_per_week
income,
<=50K,38.840048
>50K,45.452896


In [19]:
def example1_pandas(data):
    return pd.DataFrame(data.groupby('income')['hours-per-week'].mean())

In [20]:
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [21]:
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [22]:
android_device.head()

,Retail Branding,Marketing Name,Device,Model
0,NaN,NaN,AD681H,Smartfren Andromax AD681H
1,NaN,NaN,FJL21,FJL21
2,NaN,NaN,T31,Panasonic T31
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2
4,3Q,OC1020A,OC1020A,OC1020A


In [23]:
example1_pandas(data)

,hours-per-week
income,
<=50K,38.840048
>50K,45.452896


In [24]:
begin_ps = time.time()
aggr_query = """
    SELECT *
    FROM user_usage JOIN user_device on user_usage.use_id=user_device.use_id
"""
finish_ps = (time.time() - begin_ps)* 1_000
print(finish_ps, 'ms')
ps.sqldf(aggr_query, locals())

0.05125999450683594 ms


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,use_id,user_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,22787,12921,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,22788,28714,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,22789,28714,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,22790,29592,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,22792,28217,android,5.1,SM-G361F,1
5,71.59,79.26,1557.33,22793,22793,28217,android,5.1,SM-G361F,1
6,71.59,79.26,519.12,22794,22794,28217,android,5.1,SM-G361F,1
7,71.59,79.26,519.12,22795,22795,28217,android,5.1,SM-G361F,1
8,30.92,22.77,3114.67,22799,22799,29643,android,6.0,ONEPLUS A3003,1
9,69.80,14.70,25955.55,22801,22801,10976,android,4.4,GT-I9505,1


In [25]:
begin_pd = time.time()
result = pd.merge(user_usage,
                 user_device,
                 on='use_id')
finish_pd = (time.time() - begin_pd)*1_000
print(finish_pd, 'ms')
result.head()

6.021738052368164 ms


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,user_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,12921,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,28714,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,28714,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,29592,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,28217,android,5.1,SM-G361F,1


In [26]:
print('pandasql быстрее pandas в {0} раз'.format(finish_pd/finish_ps))

pandasql быстрее pandas в 117.47441860465116 раз
